# Mall Customer Segmentation Analysis

## Introduction

As a data science student, I aim to apply unsupervised learning to segment mall customers based on their demographic and behavioral data. This project uses the Mall Customer Segmentation Data from Kaggle to identify distinct customer groups, which can help mall management tailor marketing strategies. My approach includes exploratory data analysis (EDA), clustering with unsupervised learning models, and hyperparameter optimization to ensure robust results. This notebook documents my process, from data collection to model evaluation, addressing the project rubric's requirements.

### Step 1: Data Gathering and Provenance

#### Explanation

I selected the "Mall Customer Segmentation Data" from Kaggle, a publicly available dataset ideal for clustering tasks. It contains 200 records with features: CustomerID, Gender, Age, Annual Income (in thousands of dollars), and Spending Score (1-100). The data is clean, with no missing values, and sourced from Kaggle (https://www.kaggle.com/vjchoudhary7/customer-segmentation-tutorial-in-python). I downloaded the CSV file and placed it in my working directory. My goal is to use this data to segment customers without predefined labels, making it suitable for unsupervised learning.


In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('Mall_Customers.csv')

# Display the first few rows and basic information
print(data.head())
print(data.info())
print(data.describe())